In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tyler.merp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tyler.merp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
posts = pd.read_csv('AllDataFiltered.csv')
posts.head(5)

,combined_text,link_flair_text
0,"for everyone, whether you're worried about cov...",psa
1,and people wonder why the virus spreads…,rant
2,r/udub mods,meme
3,thoughts on madrona? i have an emotional suppo...,discussion
4,soc 222 anyone has took or taking soc222(socio...,academics


In [4]:
def clean_tokenize(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    return tokens

posts['tokens'] = posts['combined_text'].apply(clean_tokenize)

In [27]:
results = []

In [28]:
count_vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False, max_features=5000)

texts_bow = count_vectorizer.fit_transform(posts['tokens'])

X_train_bow, X_test_bow, y_train, y_test = train_test_split(texts_bow, posts['link_flair_text'], test_size=0.2, random_state=52)

clf_bow = MultinomialNB()
clf_bow.fit(X_train_bow, y_train)

y_pred_bow = clf_bow.predict(X_test_bow)

accuracy_bow = accuracy_score(y_test, y_pred_bow)

results.append({'vectorizationType': 'Bag of Words (BoW)', 'Dataset': 'AllDataFiltered.csv', 'Accuracy': accuracy_bow * 100})

/Users/tyler.merp/anaconda3/envs/INFO492/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [29]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False, max_features=5000)

texts_tfidf = tfidf_vectorizer.fit_transform(posts['tokens'])

X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(texts_tfidf, posts['link_flair_text'], test_size=0.2, random_state=52)

clf_tfidf = MultinomialNB()
clf_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)

results.append({'vectorizationType': 'TF-IDF', 'Dataset': 'AllDataFiltered.csv', 'Accuracy': accuracy_tfidf * 100})

/Users/tyler.merp/anaconda3/envs/INFO492/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [30]:
docs = posts["combined_text"].values
tokenized_docs = posts["tokens"].values
model = Word2Vec(sentences=tokenized_docs, window = 2, vector_size=200, workers=1, seed=52)
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(tokenized_docs, model=model)

X_train, X_test, y_train, y_test = train_test_split(vectorized_docs, posts['link_flair_text'], test_size=0.2, random_state=52)
clf = Pipeline([
    ('scaler', MinMaxScaler()),
    ('clf', MultinomialNB()),
])
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy_word2vec = accuracy_score(y_test, y_pred)

results.append({'vectorizationType': 'Word2Vec', 'Dataset': 'AllDataFiltered.csv', 'Accuracy': accuracy_word2vec * 100})

In [31]:
print(pd.DataFrame(results))

    vectorizationType              Dataset   Accuracy
0  Bag of Words (BoW)  AllDataFiltered.csv  53.519131
1              TF-IDF  AllDataFiltered.csv  48.984412
2            Word2Vec  AllDataFiltered.csv  40.576287
